In [1]:
import json, time, datetime, math, csv, copy, sys
import numpy as np
from dateutil.parser import parse
from tqdm import tqdm

In [2]:
# read worker attribute: worker_quality
worker_quality = {}
csvfile = open("worker_quality.csv", "r")
csvreader = csv.reader(csvfile)
next(csvreader)
for line in csvreader:
    if float(line[1]) > 0.0:
        worker_quality[int(line[0])] = float(line[1]) / 100.0
    else:
        worker_quality[int(line[0])] = 0.0
csvfile.close()

In [3]:
print("len:", len(worker_quality))
print("min:", min(worker_quality.values()))
print("max:", max(worker_quality.values()))
print("mean:", np.mean(list(worker_quality.values())))

len: 1807
min: 0.0
max: 1.0
mean: 0.7311178749308246


In [4]:
# # read project id
# file = open("project_list.csv", "r")
# project_list_lines = file.readlines()
# file.close()
# project_dir = "project/"
# entry_dir = "entry/"

# all_begin_time = parse("2018-01-01T0:0:0Z")
# project_id2index_dict = {}
# project_id_list = []
# project_sub_category_list = []
# project_category_list = []
# project_industry_list = []
# project_entry_count_list = []
# project_start_date_list = []
# project_deadline_list = []
# project_average_score_list = []
# project_client_feedback_list = []
# project_total_awards_and_tips_list = []
# industry_dict = {}
# sub_category_dict = {}
# category_dict = {}

# entry_info = {}
# limit = 24
# industry_list = {}
# worker_time_dict = {}
# for line in tqdm(project_list_lines):
#     line = line.strip('\n').split(',')

#     project_id = int(line[0])
#     entry_count = int(line[1])
#     if project_id not in project_id2index_dict:
#         project_id2index_dict[project_id] = len(project_id2index_dict)
#     project_id_list.append(project_id)
#     project_entry_count_list.append(entry_count)

#     file = open(project_dir + "project_" + str(project_id) + ".txt", "r")
#     htmlcode = file.read()
#     file.close()
#     text = json.loads(htmlcode)

#     sub_category = int(text["sub_category"])
#     category = int(text["category"])
#     industry = text["industry"]
    
#     if sub_category not in sub_category_dict:
#         sub_category_dict[sub_category] = len(sub_category_dict)
#     if category not in category_dict:
#         category_dict[category] = len(category_dict)
#     if industry not in industry_dict:
#         industry_dict[industry] = len(industry_dict)
    
#     project_sub_category_list.append(sub_category)
#     project_category_list.append(category)
#     project_industry_list.append(industry)
#     project_start_date_list.append(parse(text["start_date"]))
#     project_deadline_list.append(parse(text["deadline"]))
#     project_average_score_list.append(float(text["average_score"]))
#     project_client_feedback_list.append(float(text["client_feedback"]))
#     project_total_awards_and_tips_list.append(float(text["total_awards_and_tips"]))


#     entry_info[project_id] = {}
#     k = 0
#     while (k < entry_count):
#         file = open(entry_dir + "entry_" + str(project_id) + "_" + str(k) + ".txt", "r")
#         htmlcode = file.read()
#         file.close()
#         text = json.loads(htmlcode)

#         for item in text["results"]:
#             entry_number = int(item["entry_number"])
#             entry_info[project_id][entry_number] = {}
#             entry_info[project_id][entry_number]["entry_created_at"] = parse(item["entry_created_at"]) # worker answer_time
#             entry_info[project_id][entry_number]["worker"] = int(item["author"]) # work_id

#             if int(item["author"]) not in worker_time_dict:
#                 worker_time_dict[int(item["author"])] = list()
#             worker_time_dict[int(item["author"])].append(parse(item["entry_created_at"]))

#             entry_info[project_id][entry_number]["winner"] = bool(item["winner"])
#             entry_info[project_id][entry_number]["finalist"] = bool(item["finalist"])
#             entry_info[project_id][entry_number]["award_value"] = float(item["award_value"]) if item["award_value"] is not None else None
#             entry_info[project_id][entry_number]["tip_value"] = float(item["tip_value"]) if item["tip_value"] is not None else None
#             entry_info[project_id][entry_number]["score_list"] = list()
#             for revision_item in item["revisions"]:
#                 entry_info[project_id][entry_number]["score_list"].append(int(revision_item["score"]))

#         k += limit

# print("finish read_data")

In [5]:
# read project id
file = open("project_list.csv", "r")
project_list_lines = file.readlines()
file.close()
project_dir = "project/"
entry_dir = "entry/"

all_begin_time = parse("2018-01-01T0:0:0Z")

project_info = {}
entry_info = {}
answer_info = {}
limit = 24

average_score_list = []
client_feedback_list = []
total_awards_and_tips_list = []

project_id2index_dict = {}
project_index2id_dict = {}
worker_id2index_dict = {}
worker_index2id_dict = {}

industry_dict = {}
sub_category_dict = {}
category_dict = {}

worker_list = []
worker_time_list = []

for line in tqdm(project_list_lines):
    line = line.strip('\n').split(',')

    project_id = int(line[0])
    entry_count = int(line[1])

    file = open(project_dir + "project_" + str(project_id) + ".txt", "r")
    htmlcode = file.read()
    file.close()
    text = json.loads(htmlcode)

    project_info[project_id] = {}

    sub_category = int(text["sub_category"])
    category = int(text["category"])
    industry = text["industry"]
    

    if project_id not in project_id2index_dict:
        project_id2index_dict[project_id] = len(project_id2index_dict)
        project_index2id_dict[project_id2index_dict[project_id]] = project_id
    if sub_category not in sub_category_dict:
        sub_category_dict[sub_category] = len(sub_category_dict)
    if category not in category_dict:
        category_dict[category] = len(category_dict)
    if industry not in industry_dict:
        industry_dict[industry] = len(industry_dict)

    project_info[project_id]["sub_category"] = sub_category # project sub_category
    project_info[project_id]["category"] = category # project category
    project_info[project_id]["entry_count"] = int(text["entry_count"]) # project answers in total
    project_info[project_id]["start_date"] = parse(text["start_date"]) # project start date
    project_info[project_id]["deadline"] = parse(text["deadline"]) # project end date

    project_info[project_id]["average_score"] = float(text["average_score"])
    project_info[project_id]["client_feedback"] = float(text["client_feedback"])
    project_info[project_id]["total_awards_and_tips"] = float(text["total_awards_and_tips"])
    average_score_list.append(float(text["average_score"]))
    client_feedback_list.append(float(text["client_feedback"]))
    total_awards_and_tips_list.append(float(text["total_awards_and_tips"]))

    project_info[project_id]["industry"] = industry # project domain

    entry_info[project_id] = {}
    answer_info[project_id] = {}
    k = 0
    while (k < entry_count):
        file = open(entry_dir + "entry_" + str(project_id) + "_" + str(k) + ".txt", "r")
        htmlcode = file.read()
        file.close()
        text = json.loads(htmlcode)

        for item in text["results"]:
            entry_number = int(item["entry_number"])
            worker_number = int(item["author"])
            if worker_number not in worker_id2index_dict:
                worker_id2index_dict[worker_number] = len(worker_id2index_dict)
                worker_index2id_dict[worker_id2index_dict[worker_number]] = worker_number

            answer_info[project_id][worker_number] = {}
            answer_info[project_id][worker_number]["winner"] = bool(item["winner"])
            answer_info[project_id][worker_number]["finalist"] = bool(item["finalist"])
            answer_info[project_id][worker_number]["award_value"] = float(item["award_value"]) if item["award_value"] is not None else None
            answer_info[project_id][worker_number]["tip_value"] = float(item["tip_value"]) if item["tip_value"] is not None else None

            worker_list.append(item["author"])
            worker_time_list.append(parse(item["entry_created_at"]))

            entry_info[project_id][entry_number] = {}
            entry_info[project_id][entry_number]["entry_created_at"] = parse(item["entry_created_at"]) # worker answer_time
            entry_info[project_id][entry_number]["worker"] = worker_number # work_id

            entry_info[project_id][entry_number]["winner"] = bool(item["winner"])
            entry_info[project_id][entry_number]["finalist"] = bool(item["finalist"])
            entry_info[project_id][entry_number]["award_value"] = float(item["award_value"]) if item["award_value"] is not None else None
            entry_info[project_id][entry_number]["tip_value"] = float(item["tip_value"]) if item["tip_value"] is not None else None

        k += limit

worker_time_list, worker_list = zip(*sorted(zip(worker_time_list, worker_list)))

for project_id, _ in project_info.items():
    project_info[project_id]["average_score"] = (
        project_info[project_id]["average_score"] - min(average_score_list)
    ) / (max(average_score_list) - min(average_score_list))
    project_info[project_id]["client_feedback"] = (
        project_info[project_id]["client_feedback"] - min(client_feedback_list)
    ) / (max(client_feedback_list) - min(client_feedback_list))
    project_info[project_id]["total_awards_and_tips"] = (
        project_info[project_id]["total_awards_and_tips"] - min(total_awards_and_tips_list)
    ) / (max(total_awards_and_tips_list) - min(total_awards_and_tips_list))

print("finish read_data")

100%|██████████| 2501/2501 [01:07<00:00, 37.25it/s]


finish read_data


In [7]:
click_count = {}
for p_id, e_info in entry_info.items():
    for e_id, info in e_info.items():
        if info["worker"] not in click_count:
            click_count[info["worker"]] = 0
        click_count[info["worker"]] += 1
print(np.mean(list(click_count.values())))

105.6087437742114


In [15]:
np.percentile(list(click_count.values()), 75)

90.0

In [9]:
len(click_count)

1807

In [5]:
len(project_id2index_dict)

NameError: name 'project_id2index_dict' is not defined

In [6]:
print(len(category_dict))
print(len(sub_category_dict))
print(len(industry_dict))

7
29
37


In [5]:
import pickle
data = worker_quality, project_info, answer_info, \
    worker_time_list, worker_list, project_id2index_dict, project_index2id_dict, \
        worker_id2index_dict, worker_index2id_dict, industry_dict, sub_category_dict, category_dict
with open("env_data.pickle", "wb") as f:
    pickle.dump(data, f, 1)

In [6]:
project_info

{2908089: {'sub_category': 23,
  'category': 7,
  'entry_count': 99,
  'start_date': datetime.datetime(2019, 2, 2, 21, 26, 1, 604373, tzinfo=tzutc()),
  'deadline': datetime.datetime(2019, 2, 16, 21, 26, 1, 604373, tzinfo=tzutc()),
  'average_score': 0.82,
  'client_feedback': 0.5656565656565656,
  'total_awards_and_tips': 0.09420289855072464,
  'industry': 'health-care-and-biotech'},
 2909696: {'sub_category': 23,
  'category': 7,
  'entry_count': 105,
  'start_date': datetime.datetime(2019, 2, 28, 19, 40, 45, 627395, tzinfo=tzutc()),
  'deadline': datetime.datetime(2019, 3, 3, 19, 40, 45, 627395, tzinfo=tzutc()),
  'average_score': 0.72,
  'client_feedback': 0.8476190476190476,
  'total_awards_and_tips': 0.04830917874396135,
  'industry': 'physical-fitness'},
 2909401: {'sub_category': 23,
  'category': 7,
  'entry_count': 127,
  'start_date': datetime.datetime(2019, 2, 22, 21, 44, 37, 812170, tzinfo=tzutc()),
  'deadline': datetime.datetime(2019, 3, 1, 21, 44, 37, 812170, tzinfo=tzu

In [7]:
len(industry_dict)

37

In [7]:
with open('./env_data.pickle', 'rb') as f:
    data = pickle.load(f)
    (
        worker_quality,
        project_info,
        answer_info,
        worker_time_list,
        worker_list,
        project_id2index_dict,
        project_index2id_dict,
        worker_id2index_dict,
        worker_index2id_dict,
        industry_dict,
        sub_category_dict,
        category_dict
    ) = data

In [8]:
len(project_id2index_dict)

2501

In [9]:
len(project_index2id_dict)

2501

In [10]:
len(worker_quality)

1807